In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
import time

# Standard Lakebase name for DevSecOps Labs
LAKEBASE_NAME = "margie-app-lakebase"

try:
    # Initialize Databricks SDK client
    w = WorkspaceClient()
    
    # Try to get existing instance
    try:
        instance = w.database.get_database_instance(name=LAKEBASE_NAME)
        print(f"Found existing Lakebase instance: {LAKEBASE_NAME}")
    except:
        # Instance doesn't exist, create it
        print(f"Creating Lakebase instance: {LAKEBASE_NAME}")
        print("This may take 2-3 minutes...")
        instance = w.database.create_database_instance(
            DatabaseInstance(
                name=LAKEBASE_NAME,
                capacity="CU_1"
            )
        )
        print(f"Lakebase instance creation initiated: {LAKEBASE_NAME}")
    
    # Wait for instance to be available
    print("Waiting for instance to be ready...")
    max_wait = 300  # 5 minutes
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        instance = w.database.get_database_instance(name=LAKEBASE_NAME)
        
        if hasattr(instance, 'read_write_dns') and instance.read_write_dns:
            break
            
        print(f"  Status: {instance.state if hasattr(instance, 'state') else 'provisioning'}...")
        time.sleep(10)
    
    # Extract connection parameters for app configuration
    if hasattr(instance, 'read_write_dns') and instance.read_write_dns:
        lakebase_host = instance.read_write_dns
        lakebase_database = "databricks_postgres"
        enable_chat_history = "true"
        
        print(f"\nLakebase Configuration:")
        print(f"  Host: {lakebase_host}")
        print(f"  Database: {lakebase_database}")
        print(f"  Chat History: {enable_chat_history}")
    else:
        print("\nInstance is still provisioning. Check status at:")
        print(f"  Compute > Database Instances > {LAKEBASE_NAME}")
        print("Re-run this cell once instance shows 'Available'")
    
except Exception as e:
    print(f"Error with Lakebase instance: {str(e)}")

In [0]:
import os

# Get current user dynamically
current_user = spark.sql("SELECT current_user()").collect()[0][0]

# Construct dynamic path to user's deployed app
app_path = f"/Workspace/Users/{current_user}/DevSecOps_Labs/notebooks/chatbot-app"
env_file = f"{app_path}/.env.local"

# Build environment configuration
env_content = f"""# Lakebase Configuration (Auto-generated from notebook)
PGHOST={lakebase_host}
PGDATABASE={lakebase_database}
PGPORT=5432
PGSSLMODE=require

# Feature Flags
CHAT_HISTORY_ENABLED={enable_chat_history}

# Databricks Auth (uses your current session)
DATABRICKS_HOST=https://{lakebase_host}
DATABRICKS_SERVING_ENDPOINT=your-agent-endpoint
"""

# Write configuration file
with open(env_file, 'w') as f:
    f.write(env_content)

print(f"✓ Configuration written to {env_file}")
print(f"✓ App path: {app_path}")

In [0]:
# Create the database schema and tables
import subprocess
import os

os.chdir(f"{app_path}/packages/db")

print("Creating ai_chatbot schema and tables...")
result = subprocess.run(
    ["npx", "drizzle-kit", "push"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Database schema created successfully!")
    print("\nTables created:")
    print("  - ai_chatbot.Chat")
    print("  - ai_chatbot.Message")
    print("  - ai_chatbot.User")
else:
    print("✗ Migration failed:")
    print(result.stderr)

In [0]:
# Verify the database connection and schema
from databricks import sql
import os

connection = sql.connect(
    server_hostname=lakebase_host,
    http_path=f"/sql/1.0/warehouses/{lakebase_database}",  # Adjust if needed
    access_token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
)

cursor = connection.cursor()

# Check if tables exist
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'ai_chatbot'
""")

tables = cursor.fetchall()
print("✓ Database tables found:")
for table in tables:
    print(f"  - ai_chatbot.{table[0]}")

cursor.close()
connection.close()

print("\n✓ Lakebase setup complete!")
print("\nNext steps:")
print("1. Restart your chatbot app")
print("2. Chat history will now be persisted to Lakebase")

Ope! Well hello there! I'm MARGIE - that stands for Meijer Archive Responder for Grocery Incident Exploration. I'm your friendly Midwest customer service expert here to help you dig into store operations questions, wouldn'tcha know!

In [0]:
result = spark.sql("""
    SELECT devsecops_labs.default.checkpoint(
        current_user(),
        'Agent Bricks Lab: Exercise 5 Complete',
        ''
    )
""").collect()[0][0]

print(result)